In [1]:
%cd ..
%load_ext autoreload
%autoreload 2

/mnt/data4/haryoaw_workspace/projects/2021_2/mbekk/idrecibrew2


In [2]:
from idrecibrew2.data import Seq2SeqDataFactory, Seq2SeqDataFactoryArgs
from idrecibrew2.data.indonlg_tokenizer.tokenizer import IndoNLGTokenizer
from idrecibrew2.model import LitSeq2SeqTransformers, LitSeq2SeqTransformersArgs

In [3]:
from transformers import AutoTokenizer

In [9]:
tokenizer = AutoTokenizer.from_pretrained("Wikidepia/IndoT5-base")

In [7]:
tokenizer.tokenize("saya adalah pengembala sapi!")

['▁saya', '▁adalah', '▁pen', 'gembala', '▁sapi', '!']

In [8]:
tokenizer.vocab_size

32100

In [29]:
args_seq = Seq2SeqDataFactoryArgs(tokenizer=tokenizer)

In [30]:
df =Seq2SeqDataFactory(args_seq)

In [31]:
dl = df.produce_dataloader_from_csv("data/processed/test.csv")

  0%|          | 0/2 [00:00<?, ?ba/s]

In [32]:
rslt = next(iter(dl))

In [33]:
rslt

{'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 0]]), 'input_ids': tensor([[16689,  2322,   858,  1750,     1],
        [16689, 25237,  3434,     1,     0]]), 'labels': tensor([[   86,   749, 16689,    11,     4, 16296,  5555,   858,   228,    11,
            75,   721, 14856,    11,     5,    11,     2,  7846,    11,    39,
            11,     2,   237,   980,   686,  4439,  1074,    11,     4,  2293,
         11961,    11,     5,    11,     2,   173,   980,   686,  4439,   950,
            11,     4,  2293, 11961,    11,     5,    11,     2,    78, 11638,
          3801,    11,     7,  5448,  5105,    11,     4,  7298,    19,   200,
           165,   158,   223,   190,    11,     5,    11,     2,    86,   165,
           158,   223,  3191,    11,     2,    78,   165,  8029,  3434,    11,
             2,   279,   165,  8029, 14340,  4066,    11,     2,   285,   749,
         25237,  1074,    11,     4, 28255,    11,     8,  1535,  3576,  1655,
           359,    11,  

In [82]:
lit_model_args = LitSeq2SeqTransformersArgs(
    vocab_size=tokenizer.vocab_size, model_type="Wikidepia/IndoT5-base"
)
lit_model = LitSeq2SeqTransformers(config=lit_model_args)

In [ ]:
lit_model.generate("")

In [75]:
tokenizer.decode([30827])

'@'

In [77]:
tokenizer(" @@ ")

{'input_ids': [12281, 30827, 11, 1], 'attention_mask': [1, 1, 1, 1]}

## Check pretrained model

In [84]:
recipe_model = LitSeq2SeqTransformers.load_from_checkpoint("outputs/indo-t5/model-epoch=27-val_loss=1.287.ckpt")

In [170]:
input_to_dl = tokenizer("sop ayam", return_tensors="pt")

In [181]:
result = tokenizer.decode(recipe_model.model.generate(input_to_dl['input_ids'], num_beams=1, max_length=400, top_k=1, repetition_penalty=1.1, do_sample=True)[0])

In [182]:
result

'<pad> 1 / 2 kg ayam @@ 3 siung bawang putih @@ 4 siung bawang merah @@ 1 ruas jahe @@ 1 sdt ketumbar bubuk @@ secukupnya garam @@ secukupnya gula pasir @@ secukupnya air @@ minyak untuk menumis @@ </s>'

## Processed for T5

In [66]:
import pandas as pd

In [81]:
for data in ["train", "dev", "test"]:
    df = pd.read_csv(f"data/processed/{data}.csv")
    df['tgt'] = df.tgt.str.replace("||", "@@", regex=False)
    df.to_csv(f"data/processed/{data}_t5.csv", index=False)